In [5]:
import numpy as np
import scipy.linalg as la

def prewhiten(X):
    """
    Prewhiten the data matrix X so that its rows have a sample mean of zero
    and a sample covariance matrix that is the identity matrix.
    """
    # Subtract the mean of each row
    X_mean = X.mean(axis=1, keepdims=True)
    X_centered = X - X_mean
    
    # Compute the covariance matrix
    cov_matrix = np.cov(X_centered)
    
    # Compute the whitening matrix using the Cholesky decomposition of the inverse covariance matrix
    whitening_matrix = la.inv(la.cholesky(cov_matrix)).T
    
    # Apply the whitening matrix to X_centered
    X_whitened = whitening_matrix @ X_centered
    
    return X_whitened, whitening_matrix

def compute_cumulants(X):
    """
    Compute the fourth-order cumulants of the data matrix X.
    """
    m, n = X.shape
    cumulants = np.zeros((m, m))
    for i in range(m):
        for j in range(m):
            if i == j:
                # Fourth cumulant for i == j is the kurtosis minus 3 (since data is standardized)
                cumulants[i, j] = np.mean(X[i] ** 4) - 3
            else:
                # For i != j, the cumulant is the joint cumulant
                cumulants[i, j] = np.mean(X[i] ** 2 * X[j] ** 2) - (
                    np.mean(X[i] ** 2) * np.mean(X[j] ** 2)
                )
    return cumulants


# Correcting the implementation of the optimization function


def optimize_contrast_function(X):
    """
    Optimize a contrast function to find the rotation matrix.
    Here we use a simple form of contrast function based on the fourth cumulant (kurtosis).
    The goal is to maximize non-Gaussianity which is measured by the absolute value of kurtosis.
    """
    m, n = X.shape

    # Initialize the rotation matrix to identity
    rotation_matrix = np.eye(m)

    # Define the learning rate
    learning_rate = 0.1

    # Perform gradient ascent
    for _ in range(100):  # Number of iterations
        # Compute the cumulants for the rotated data
        cumulants = compute_cumulants(rotation_matrix.T @ X)
        # Update rule: rotation_matrix += learning_rate * gradient
        # The gradient is approximated by the cumulants since we aim to maximize them
        rotation_matrix += learning_rate * cumulants @ rotation_matrix
        # Re-orthogonalize the rotation matrix
        u, _, vh = la.svd(rotation_matrix, full_matrices=False)
        rotation_matrix = u @ vh

    return rotation_matrix

In [6]:
# Since we don't have the actual data matrix X, let's define a dummy X with random values for demonstration purposes.
# In a real case, X should be your prewhitened data matrix.
np.random.seed(0)  # For reproducibility
m, n = 5, 1000  # m variables, n observations
np.random.seed(0)
X_original = np.random.randn(m, n)  # Original data

# Prewhiten the data
X_prewhitened, whitening_matrix = prewhiten(X_original)

In [7]:

rotation_matrix = optimize_contrast_function(X_prewhitened)

# Compute the estimated source components Z
Z = rotation_matrix.T @ X_prewhitened

rotation_matrix, Z

(array([[ 1.00000000e+00, -1.70035502e-16,  2.76795099e-15,
          2.31777719e-18, -8.96265672e-16],
        [ 2.87610313e-16,  1.00000000e+00, -3.88298838e-15,
         -3.39842914e-16, -5.05268049e-15],
        [-2.70662047e-15,  3.85184982e-15,  1.00000000e+00,
         -6.47702444e-15, -2.57643269e-15],
        [-2.80571246e-16, -6.73765627e-17,  6.27773661e-15,
          1.00000000e+00,  6.11114438e-15],
        [ 8.53039354e-16,  4.26604467e-15,  3.22979571e-15,
         -5.87034157e-15,  1.00000000e+00]]),
 array([[ 1.83216152,  0.45103971,  1.03692825, ...,  0.14121032,
         -1.11627752, -0.31680891],
        [ 0.61793684,  0.9219753 , -0.41757864, ...,  0.15403048,
         -1.22870519, -1.37813131],
        [-1.48304651, -1.72227348,  0.14040634, ..., -0.14238806,
         -1.26174358, -0.04526549],
        [ 1.57784682,  0.57775505, -0.09644962, ..., -0.01369647,
          1.61546956, -0.75393087],
        [ 0.66152738,  1.9276069 ,  0.19602298, ...,  0.91394239,
    